Modelo para anlisis de sentmientos para luego precedir si tiene sintomas de depresion.

In [1]:
from string import punctuation
import pandas as pd
import nltk
import re

In [2]:
# read the data from tweets_public.csv and create a dataframe
df = pd.read_csv('data/tweets_public.csv', encoding='utf-8')
df.head()

,airline_sentiment,is_reply,reply_count,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,neutral,False,0,0,Trabajar en #Ryanair como #TMA: https://t.co/r...,NaN,Fri Nov 03 12:05:12 +0000 2017,926419989107798016,NaN,Madrid
1,neutral,True,0,0,@Iberia @FIONAFERRER Cuando gusten en Cancún s...,NaN,Sun Nov 26 18:40:28 +0000 2017,934854385577943041,NaN,Mexico City
2,negative,False,0,0,Sabiais que @Iberia te trata muy bien en santi...,NaN,Mon Dec 25 15:40:45 +0000 2017,945318406441635840,NaN,Madrid
3,negative,False,0,0,NUNCA NUNCA NUNCA pidáis el café de Ryanair.\n...,NaN,Mon Nov 06 14:18:35 +0000 2017,927540721296568320,NaN,Pacific Time (US & Canada)
4,positive,True,0,0,@cris_tortu @dakar @Iberia @Mitsubishi_ES @BFG...,NaN,Mon Jan 01 23:00:57 +0000 2018,947965901332197376,NaN,Buenos Aires


In [3]:
df.shape

(7867, 10)

# Preprocesamiento
Limpieza de datos

In [4]:
# create a new dataframe with only the text and airline_sentiment columns and tweet id with the name df_sentiment
df_sentiment = df[['text', 'airline_sentiment', 'tweet_id']]

In [5]:
# transform the text letters to lowercase
df_sentiment['text'] = df_sentiment['text'].str.lower()
df_sentiment.head()

C:\Users\raula\AppData\Local\Temp\ipykernel_7712\3390815067.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment['text'] = df_sentiment['text'].str.lower()


,text,airline_sentiment,tweet_id
0,trabajar en #ryanair como #tma: https://t.co/r...,neutral,926419989107798016
1,@iberia @fionaferrer cuando gusten en cancún s...,neutral,934854385577943041
2,sabiais que @iberia te trata muy bien en santi...,negative,945318406441635840
3,nunca nunca nunca pidáis el café de ryanair.\n...,negative,927540721296568320
4,@cris_tortu @dakar @iberia @mitsubishi_es @bfg...,positive,947965901332197376


In [6]:
# Referencia: https://stackoverflow.com/questions/6718633/python-regular-expression-again-match-url
# remove the urls from the text but keep all the text after the url
df_sentiment.loc[:, 'text'] = df_sentiment['text'].apply(lambda x: re.split('http[s]*\S+', str(x))[0])
df_sentiment.head()

,text,airline_sentiment,tweet_id
0,trabajar en #ryanair como #tma:,neutral,926419989107798016
1,@iberia @fionaferrer cuando gusten en cancún s...,neutral,934854385577943041
2,sabiais que @iberia te trata muy bien en santi...,negative,945318406441635840
3,nunca nunca nunca pidáis el café de ryanair.\n...,negative,927540721296568320
4,@cris_tortu @dakar @iberia @mitsubishi_es @bfg...,positive,947965901332197376


In [7]:
# remove the punctuation from the text
df_sentiment.loc[:, 'text'] = df_sentiment['text'].apply(lambda x: ''.join(c for c in x if c not in punctuation))
df_sentiment.head()

,text,airline_sentiment,tweet_id
0,trabajar en ryanair como tma,neutral,926419989107798016
1,iberia fionaferrer cuando gusten en cancún se ...,neutral,934854385577943041
2,sabiais que iberia te trata muy bien en santia...,negative,945318406441635840
3,nunca nunca nunca pidáis el café de ryanair\nb...,negative,927540721296568320
4,cristortu dakar iberia mitsubishies bfgoodrich...,positive,947965901332197376


In [8]:
# change the \n to a space
df_sentiment.loc[:, 'text'] = df_sentiment['text'].apply(lambda x: x.replace('\n', ' '))
df_sentiment.head()

,text,airline_sentiment,tweet_id
0,trabajar en ryanair como tma,neutral,926419989107798016
1,iberia fionaferrer cuando gusten en cancún se ...,neutral,934854385577943041
2,sabiais que iberia te trata muy bien en santia...,negative,945318406441635840
3,nunca nunca nunca pidáis el café de ryanair bu...,negative,927540721296568320
4,cristortu dakar iberia mitsubishies bfgoodrich...,positive,947965901332197376


In [9]:
# remove the stopwrods from the text
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('spanish'))
df_sentiment.loc[:, 'text'] = df_sentiment['text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
df_sentiment.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\raula\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,airline_sentiment,tweet_id
0,trabajar ryanair tma,neutral,926419989107798016
1,iberia fionaferrer gusten cancún viaja disfrut...,neutral,934854385577943041
2,sabiais iberia trata bien santiago chilete cam...,negative,945318406441635840
3,nunca nunca nunca pidáis café ryanair bueno ve...,negative,927540721296568320
4,cristortu dakar iberia mitsubishies bfgoodrich...,positive,947965901332197376


In [10]:
# remove the emojis from the text

# regular expression pattern to remove emojis from text
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # faces
        u"\U0001F300-\U0001F5FF"  # simbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        "]+", flags=re.UNICODE)

# use lambda function to remove the emojis from the text
df_sentiment.loc[:, 'text'] = df_sentiment['text'].apply(lambda x: emoji_pattern.sub(r'', x))
df_sentiment.head()

,text,airline_sentiment,tweet_id
0,trabajar ryanair tma,neutral,926419989107798016
1,iberia fionaferrer gusten cancún viaja disfrut...,neutral,934854385577943041
2,sabiais iberia trata bien santiago chilete cam...,negative,945318406441635840
3,nunca nunca nunca pidáis café ryanair bueno ve...,negative,927540721296568320
4,cristortu dakar iberia mitsubishies bfgoodrich...,positive,947965901332197376


In [11]:
# remove the numbers from the text
df_sentiment.loc[:, 'text'] = df_sentiment['text'].apply(lambda x: re.sub(r'\d+', '', x))
df_sentiment.head()

,text,airline_sentiment,tweet_id
0,trabajar ryanair tma,neutral,926419989107798016
1,iberia fionaferrer gusten cancún viaja disfrut...,neutral,934854385577943041
2,sabiais iberia trata bien santiago chilete cam...,negative,945318406441635840
3,nunca nunca nunca pidáis café ryanair bueno ve...,negative,927540721296568320
4,cristortu dakar iberia mitsubishies bfgoodrich...,positive,947965901332197376


In [12]:
df_sentiment = df[['text', 'airline_sentiment']]
# df_sentiment['label'] = df_sentiment['airline_sentiment'].apply(lambda x: '1 start' if x == 'negative' else '5 stars')
df_sentiment['label'] = df_sentiment['airline_sentiment'].apply(lambda x: 0 if x == 'negative' else 1)
df_sentiment = df_sentiment[['text', 'label']]
df_sentiment

C:\Users\raula\AppData\Local\Temp\ipykernel_7712\4261467530.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment['label'] = df_sentiment['airline_sentiment'].apply(lambda x: 0 if x == 'negative' else 1)


,text,label
0,Trabajar en #Ryanair como #TMA: https://t.co/r...,1
1,@Iberia @FIONAFERRER Cuando gusten en Cancún s...,1
2,Sabiais que @Iberia te trata muy bien en santi...,0
3,NUNCA NUNCA NUNCA pidáis el café de Ryanair.\n...,0
4,@cris_tortu @dakar @Iberia @Mitsubishi_ES @BFG...,1
...,...,...
7862,@Iberia @iberiaexpress especialistas en dejart...,0
7863,"Con @Iberia, mi destino a un solo click. ¡Dese...",1
7864,@Iberia Muy bien. Muchas gracias,1
7865,Es que volar con Ryanair es peor que irte a ch...,0


In [13]:
# save the dataframe to a csv file
df_sentiment.to_csv('data/clean_tweets_sentiment.csv', index=False)

# Hugging Face

In [14]:
model_preload1 = "nlptown/bert-base-multilingual-uncased-sentiment"
# model_preload = "distilbert-base-uncased"
# model_preload = "xlm-roberta-base"
# model_preload = "distilbert-base-multilingual-cased"
# model_preload = "bert-base-multilingual-cased"
model_preload =  'dccuchile/bert-base-spanish-wwm-cased'
# model_preload = "RoBERTa-base"

In [15]:

data = ["Amo el mundo en el que vivo", "Odio a todas las personas que conozco"]

In [16]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9905071258544922},
 {'label': 'NEGATIVE', 'score': 0.9892126321792603}]

In [17]:
sentiment_pipeline('no estoy normal')

[{'label': 'NEGATIVE', 'score': 0.9994358420372009}]

In [18]:
specific_model = pipeline("sentiment-analysis", model=model_preload1)
specific_model(data)

[{'label': '5 stars', 'score': 0.7264281511306763},
 {'label': '5 stars', 'score': 0.6389875411987305}]

In [19]:
specific_model('estoy normal')

[{'label': '3 stars', 'score': 0.3511374890804291}]

In [20]:
specific_model('estoy triste')

[{'label': '1 star', 'score': 0.5364329814910889}]

## Fine tuning

In [21]:
# import torch
# torch.cuda.is_available()

In [22]:
# torch.cuda.empty_cache()

In [23]:
from datasets import load_dataset
local_train = load_dataset("csv", data_files="data/clean_tweets_sentiment.csv", split="train[:70%]")
local_test = load_dataset("csv", data_files="data/clean_tweets_sentiment.csv", split="train[70%:]")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/raula/.cache/huggingface/datasets/csv/default-166099f8577deca8/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


Found cached dataset csv (C:/Users/raula/.cache/huggingface/datasets/csv/default-166099f8577deca8/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


In [24]:
print(local_train.shape)
print(local_test.shape)
print(local_train[0])
print(local_test[0])

(5507, 2)
(2360, 2)
{'text': 'Trabajar en #Ryanair como #TMA: https://t.co/ruUArBe1tO #empleo', 'label': 1}
{'text': '@radioledonline @Iberia @lauraluzo Sr. Gallego, por qué no pagáis las indemnizaciones por cancelaciones? Aviación C… https://t.co/pC3cvISmjH', 'label': 0}


In [25]:
small_train_dataset = local_train.shuffle(seed=42).select(
    list(list(range(300)))
)
small_test_dataset = local_test.shuffle(seed=42).select(list(list(range(100))))
# # split df_sentiment in 70% train and 30% test
# from sklearn.model_selection import train_test_split

# train, test = train_test_split(df_sentiment, test_size=0.3, random_state=42)
# train.shape, test.shape

In [26]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_preload)

In [27]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)

# user the function preprocess_function to tokenize the train and test data
# tokenized_train = train['text'].apply(lambda x: tokenizer(x, truncation=True))
# tokenized_test = test['text'].apply(lambda x: tokenizer(x, truncation=True))
tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [28]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## train the model

In [44]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_preload, num_labels=2, id2label={0: "negative", 1: "positive"})


Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

In [46]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [31]:
from huggingface_hub import notebook_login
notebook_login()

In [47]:
from transformers import TrainingArguments, Trainer
 
repo_name = "raulangelj/huggingface_sentiment_analysis"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=25,
   per_device_eval_batch_size=25,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

c:\Users\raula\Documents\RAUL_ANGEL\UVG_COMPU\NOVENO_SEMESTRE\DISENO-E-INOVACION\analisis_sentimientos_depresion\raulangelj/huggingface_sentiment_analysis is already a clone of https://huggingface.co/raulangelj/huggingface_sentiment_analysis. Make sure you pull the latest changes with `repo.git_pull()`.


In [48]:
trainer.train()

C:\Users\raula\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/24 [00:00<?, ?it/s]

{'train_runtime': 37.0528, 'train_samples_per_second': 16.193, 'train_steps_per_second': 0.648, 'train_loss': 0.648597240447998, 'epoch': 2.0}


TrainOutput(global_step=24, training_loss=0.648597240447998, metrics={'train_runtime': 37.0528, 'train_samples_per_second': 16.193, 'train_steps_per_second': 0.648, 'train_loss': 0.648597240447998, 'epoch': 2.0})

In [49]:
trainer.evaluate()

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.6742175221443176,
 'eval_accuracy': 0.56,
 'eval_f1': 0.6451612903225805,
 'eval_runtime': 1.9528,
 'eval_samples_per_second': 51.209,
 'eval_steps_per_second': 2.048,
 'epoch': 2.0}

In [51]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.
To https://huggingface.co/raulangelj/huggingface_sentiment_analysis
   f00fff2..166666a  main -> main



In [52]:
data

['Amo el mundo en el que vivo', 'Odio a todas las personas que conozco']

In [53]:
from transformers import pipeline
 
sentiment_model = pipeline(model=repo_name)
sentiment_model(data)

[{'label': 'positive', 'score': 0.5367423892021179},
 {'label': 'negative', 'score': 0.5140738487243652}]

In [54]:
sentiment_model('no me siento bien')

[{'label': 'negative', 'score': 0.5318524837493896}]

In [55]:
sentiment_model('hoy es el mejor dia de mi vida')

[{'label': 'positive', 'score': 0.5589547753334045}]

In [56]:
sentiment_model('ya no puedo con mi vida, no quiero que me hablen')

[{'label': 'negative', 'score': 0.5341178178787231}]

# Funcion a utilizar en proyecto

In [8]:
import unicodedata
from transformers import pipeline

def has_first_person_pronouns(text):
    first_person_pronouns = ["yo", "me", "mi", "conmigo"]
    
    words = text.split()
    for word in words:
        normalize_word = remove_accent(word.lower())
        if normalize_word in first_person_pronouns:
            return True
    
    return False

def remove_accent(word):
    return ''.join((c for c in unicodedata.normalize('NFD', word) if unicodedata.category(c) != 'Mn'))

# ? should we use this?
def has_depression_words(text):
    depression_words = [
        "depresion", "tristeza", "desesperanza", "desmotivacion", "soledad", "ansiedad",
        "desgano", "desanimo", "desaliento", "abatimiento", "melancolia", "desolacion",
        "apatia", "angustia", "pesimismo", "desesperacion", "desamparo", "desconsuelo",
        "agotamiento", "cansancio", "desinteres", "insomnio", "culpa", "llanto", "suicidio",
        "autolesion", "desorden", "trastorno", "descontrol", "vacio", "apetito", "fatiga",
        "inutilidad", "retroceso", "preocupacion", "retraimiento", "negatividad", "desvalorizacion",
        "desesperado", "desesperanza", "perdida", "esperanza", "autoestima", "autoestima",
        "irritabilidad", "decision", "concentracion", "motivacion", "indefension", "vida",
        "conexion", "pena", "abandono", "inseguridad", "desapego", "agonia", "miedo", "rumiacion",
        "tratamiento", "terapia", "psicologo", "psiquiatra", "medicacion", "cura", "superacion",
        "autoayuda", "apoyo", "rehabilitacion"
    ]
    
    words = text.split()
    for word in words:
        normalize_word = remove_accent(word.lower())
        if normalize_word in depression_words:
            return True
    
    return False

def has_possible_depression_sintom(text):
    # To know if has depression sintoms, we will check 3 things:
    # 1. If the text has first person pronouns
    # 2. If the text has negative sentiment
    # 3. If the text has words related to depression
    # If the text has the 3 things, we will say that the text has depression sintoms
    # If the text has 2 of the 3 things, we will say that the text has possible depression sintoms
    # If the text has 1 of the 3 things, we will say that the text has no depression sintoms
    sentiment_model = pipeline(model="raulangelj/huggingface_sentiment_analysis")
    is_negative_sentiment = sentiment_model(text)[0]['label'] == 'negative'
    # sentiment_model(data)
    is_depression_words = has_depression_words(text)
    is_first_person_pronouns = has_first_person_pronouns(text)
    # if all the conditions are true, then the text has depression sintoms
    if is_negative_sentiment and is_depression_words and is_first_person_pronouns:
        # return 'depression'
        return True
    # if 2 of the 3 conditions are true, then the text has possible depression sintoms
    # if (is_negative_sentiment and is_depression_words) or (is_negative_sentiment and is_first_person_pronouns) or (is_depression_words and is_first_person_pronouns):
    #     # return 'possible_depression'
    #     return True
    # if 1 of the 3 conditions are true, then the text has no depression sintoms
    if is_negative_sentiment or is_depression_words or is_first_person_pronouns:
        # return 'no_depression'
        return False
    # if none of the conditions are true, then the text has no depression sintoms
    # return 'no_depression'
    return False

In [9]:
example_text_depression = "Últimamente, me cuesta encontrar motivación para levantarme de la cama por las mañanas. Siento una tristeza constante que no puedo sacudir, como si estuviera atrapado en una nube gris. Incluso las actividades que solían traerme alegría ahora me resultan abrumadoras y sin sentido. Me siento agotado todo el tiempo, tanto física como emocionalmente. La soledad se ha convertido en mi compañera constante, y me resulta difícil conectar con los demás. No puedo evitar sentir que he perdido el control de mi vida y que no hay esperanza de que las cosas mejoren. Estos pensamientos negativos y la sensación de vacío me invaden, y a veces me encuentro llorando sin razón aparente. Me preocupa que estos sentimientos persistan y no sé qué hacer para superar esta desesperanza abrumadora."
example_text_no_depression = "El sol brilla radiante en el cielo azul mientras disfruto de un paseo por el parque. Río y converso con mis amigos, sintiéndome lleno de energía y entusiasmo. Me siento agradecido por las bendiciones en mi vida y tengo metas emocionantes que persigo con determinación. Cada día es una oportunidad para crecer, aprender y disfrutar de las pequeñas cosas que hacen que la vida sea maravillosa. Me rodeo de personas positivas que me apoyan y me animan en mis sueños. La vida es un viaje emocionante y estoy emocionado de ver lo que el futuro me depara."
example_text_no_depression_sad = "Atravesé un momento difícil en mi vida recientemente. Experimenté una pérdida personal significativa que me dejó con el corazón apesadumbrado. Durante un tiempo, sentí una profunda tristeza y una sensación de vacío en mi interior. Sin embargo, me permití sentir y procesar estas emociones, encontrando consuelo en el amor y el apoyo de mis seres queridos. Poco a poco, comencé a encontrar la paz dentro de mí y a enfocarme en las cosas positivas que todavía tengo en mi vida. Aprendí a apreciar más cada momento y a encontrar la fortaleza para seguir adelante. Aunque todavía tengo momentos de tristeza, también encuentro alegría y esperanza en las pequeñas cosas de la vida. La adversidad me ha enseñado a ser más resiliente y a valorar cada experiencia, tanto positiva como negativa."
print(has_possible_depression_sintom(example_text_depression))
print(has_possible_depression_sintom(example_text_no_depression))
print(has_possible_depression_sintom(example_text_no_depression_sad))

True
False
False
